# Simple Reinforcement Learning with Tensorflow: Part 2 - Policy-based Agents
* word에 대한 observation을 한 다음에 현재 뿐만 아니라 장기적 관점에서 optimal reward를 얻을 수 있는 action을 취하는 agent를 만들어보자!
    * 이를 통해 full reinforcement agent를 만들어 낼 수 있다.
* 이런 environment를 Markov Decision Process(MDP)라고 한다.
    * 주어진 action에 대해 reward와 state 변화를 제공할 수 있을 뿐만 아니라 environment의 state와 그 state에서 agent가 취한 action의 reward도 얻을 수 있다.
    * MDP는 모든 가능한 state들의 집합 S와 모든 가능한 action들의 집합 A로 구성된다.
        * state-action pair (s, a)에 대해
            * 새로운 state s'에 대한 transition probability는 T(s, a)로 정의된다.
            * reward r는 R(s, a)를 통해 계산된다.
        * 즉, MDP에서 state s와 action a가 주어졌을 때 agent는 새로운 state s'와 reward r을 가지게 된다.
* eg) opening a door
    * 환경에 대한 정보
        * state: 우리가 보고 있는 문의 면 & 우리가 바라보고 있는 방향 & 문의 위치
        * action: 우리가 취할 수 있는 모든 행동
        * reward: 성공적으로 문을 연 경우
    * 문으로 걸어가는 행동은 문제를 해결하기 위해 중요하지만 reward를 주는 행동은 아님.
    * 이 경우 agent는 reward에 도달하기 까지 어떤 action들을 취할지에 대한 학습을 수행해야 한다.
    
## Cart-Pole Task
* 넘어지지않고 pole의 균형을 최대한 맞춰야 한다.
* two-armed bandit과는 다르게 다음과 같은 task들이 필요하다:
    * Observations:
        * pole이 현재 어디에 위치해 있는지, 균형이 맞춰졌을 때의 각도는 어떻게 되는지를 알아야 한다.
        * 이를 위해 network는 observation을 해 action의 probability를 생성해낼 때 이를 사용한다.
    * Delayed reward
        * pole이 균형을 잡고 있다는 것은 현재 뿐만 아니라 미래에도 advantageous하다는 의미
        * 이를 위해 시간에 따른 action의 가중치를 부여하는 함수를 이용해 각 observation-action pair에 대한 reward value를 조정해 나갈 것이다.
* 시간에 따른 reward를 고려하기 위해 이전에 사용했던 Policy Gradient을 약간 수정한다.
    * 각 시각에 하나 이상의 experience를 이용해 업데이트를 수행한다.
        * 버퍼에 experience를 저장해놓고 agent를 업데이트할 때 사용한다.
        * 이런 experience의 시퀀스를 rollouts 또는 experience traces라고 부른다.
            * 이를 통해 immediate reward 뿐만 아니라 그 뒤에 뒤따를 reward도 함께 고려할 수 있다.
            
## Simple Reinforcement Learning in Tensorflow Part 2-b:
### Vanilla Policy Gradient Agent
    """
    Description:
        A pole is attached by an un-actuated joint to a cart, which moves along
        a frictionless track. The pendulum starts upright, and the goal is to
        prevent it from falling over by increasing and reducing the cart's
        velocity.
    Source:
        This environment corresponds to the version of the cart-pole problem
        described by Barto, Sutton, and Anderson
    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Cart Position             -4.8                    4.8
        1       Cart Velocity             -Inf                    Inf
        2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        3       Pole Angular Velocity     -Inf                    Inf
    Actions:
        Type: Discrete(2)
        Num   Action
        0     Push cart to the left
        1     Push cart to the right
        Note: The amount the velocity that is reduced or increased is not
        fixed; it depends on the angle the pole is pointing. This is because
        the center of gravity of the pole increases the amount of energy needed
        to move the cart underneath it
    Reward:
        Reward is 1 for every step taken, including the termination step
    Starting State:
        All observations are assigned a uniform random value in [-0.05..0.05]
    Episode Termination:
        Pole Angle is more than 12 degrees.
        Cart Position is more than 2.4 (center of the cart reaches the edge of
        the display).
        Episode length is greater than 200.
        Solved Requirements:
        Considered solved when the average return is greater than or equal to
        195.0 over 100 consecutive trials.
    """

In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

try:
    xrange = xrange
except:
    xrange = range

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

In [3]:
env = gym.make('CartPole-v0')

In [4]:
"""
The Policy-Based Agent
"""
gamma = 0.99

def discount_rewards(r):
    """1D float array of reward를 만들고 discounted reward를 계산한다."""
    discounted_r =np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
        
    return discounted_r

In [10]:
class agent():
    def __init__(self, lr, s_size, a_size, h_size):
        # network의 feed-forward 부분
        # agent는 state를 이용해 action을 선택한다.
        self.state_in = tf.placeholder(shape=[None, s_size], dtype = tf.float32)
        hidden = slim.fully_connected(self.state_in, h_size, biases_initializer = None,
                                     activation_fn = tf.nn.relu)
        self.output = slim.fully_connected(hidden, a_size, activation_fn = tf.nn.softmax,
                                          biases_initializer = None)
        self.chosen_action = tf.argmax(self.output, 1)
        
        # training
        # reward를 네트워크에 feed하고 action을 선택한다.
        self.reward_holder = tf.placeholder(shape=[None], dtype = tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype = tf.int32)
        
        self.indexes =tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
        
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) * self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders= []
        
        for idx, var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32, name = str(idx) + '_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss, tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate = lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, tvars))

In [14]:
"""
Training the Agent
"""
tf.reset_default_graph()

# action: left or right
# state: [0, 1, 2, 3]
myAgent = agent(lr=1e-2, s_size=4, a_size=2, h_size=8)

total_episodes = 5000
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    total_reward = []
    total_length = []
    
    gradBuffer = sess.run(tf.trainable_variables())
    for ix, grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while(i < total_episodes):
        s = env.reset()
        running_reward = 0
        ep_history = []
        
        for j in range(max_ep):
            # 네트워크 output에 기반해 확률적으로 action을 선택한다.
            a_dist = sess.run(myAgent.output, feed_dict = {myAgent.state_in: [s]})
            a = np.random.choice(a_dist[0], p = a_dist[0])
            a = np.argmax(a_dist == a)
        
            # 해당 action을 취했을 때 얻는 reward
            s1, r, d, _ = env.step(a)
            ep_history.append([s, a, r, s1])
            s = s1
            running_reward += r

            if d == True:
                # 네트워크 업데이트
                ep_history = np.array(ep_history)
                ep_history[:, 2] = discount_rewards(ep_history[:, 2])
                feed_dict = {myAgent.reward_holder: ep_history[:, 2], 
                             myAgent.action_holder: ep_history[:, 1],
                             myAgent.state_in: np.vstack(ep_history[:, 0])}
                grads = sess.run(myAgent.gradients, feed_dict = feed_dict)

                for idx, grad in enumerate(grads):
                    gradBuffer[idx] += grad

                if i % update_frequency == 0 and i != 0:
                    feed_dict = dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict = feed_dict)

                    # 버퍼 초기화
                    for ix, grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0

                total_reward.append(running_reward)
                total_length.append(j)

                break
            
        if(i%100 == 0):
            print(np.mean(total_reward[-100:]))
            
        i+=1

14.0
25.18
27.98
32.66
45.06
49.85
58.66
85.73
118.7
117.13
149.15
153.47
154.68
152.65
151.99
162.48
176.78
183.56
181.64
179.36
175.42
188.11
193.28
191.68
192.93
190.9
191.16
194.34
196.96
198.37
198.94
197.49
198.63
197.54
199.65
200.0
198.83
196.87
194.04
196.26
197.05
194.21
196.46
193.13
187.85
185.52
187.71
193.46
195.02
188.53
